In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys, os
import matplotlib.pyplot as plt
import random
import pickle as pkl

sys.path.append(os.path.abspath('../util'))
# sys.path.append(os.path.abspath('../'))

%matplotlib inline

In [2]:
data_path = Path('../data/')
model_path = Path('../model/')
# data_genre = data_path / Path('data_w_genres.csv')
df_path= data_path / Path('df_cleaned__by_artist.csv')

CSV file contains float columns as features and using artist as a label for multiclass prediction.
If a song was produced by >1 artist the row has been split into mulitple rows so that each row only
has one artist.

In [3]:
df = pd.read_csv(df_path,index_col=0)
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,artists
0,0.998996,0.716599,0.028442,0.1950,0.0,0.563,0.909091,0.1510,0.745000,1.0,0.0,0.052219,0.485348,0.7790,Carl Woitschach
1,0.997992,0.383603,0.051316,0.0135,0.0,0.901,0.727273,0.0763,0.494026,1.0,0.0,0.047678,0.344019,0.0767,Robert Schumann
2,0.997992,0.383603,0.051316,0.0135,0.0,0.901,0.727273,0.0763,0.494026,1.0,0.0,0.047678,0.344019,0.0767,Vladimir Horowitz
3,0.606426,0.758097,0.018374,0.2200,0.0,0.000,0.454545,0.1190,0.627609,0.0,0.0,0.958720,0.439086,0.8800,Seweryn Goszczyński
4,0.998996,0.790486,0.032538,0.1300,0.0,0.887,0.090909,0.1110,0.708887,0.0,0.0,0.095562,0.442470,0.7200,Francisco Canaro


In [4]:
artists_names = df.artists

In [5]:
index_artist = dict(((i, artist) for i,artist in  enumerate(artists_names.unique())))
artist_index = dict((val, key) for key,val in index_artist.items() )

In [6]:
df = df.drop(['artists'], axis =1)

In [7]:
index_artist[2176] , artist_index['Queen']

('Queen', 2176)

# Model

- Create a generator with True/ Fake samples
    - Fake samples: Choose a random vector and assign a label that is not its true artist label (1/-1)
- Create an embedding layer for the artists and pass the remaining features to a FC layer
- Train the model to see if it can detect fakes (BCE)
- User inputs song
    - Featurize the song using spotify api
    - Input the features into model(Artist in dict | Artist not in dict)
    - Works for Artist in dict, if artist not in dict???

In [8]:
artists_names[5000], artist_index[artists_names[5000]], index_artist[2032]

('Paul McCartney', 2032, 'Paul McCartney')

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, df, cols, is_truth = 1):
        'Initialization'
        self.df = df
#         self.false_pct = false_pct
        self.cols = cols
        self.is_truth = is_truth

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

    def __getitem__(self, index):
        'Generates one sample of data'
        true_label = 1
        fake_label = -1
        
        if self.is_truth: 
            return torch.tensor([self.df[self.cols].iloc[index]]).double(), \
                   artist_index[artists_names[index]],\
                   torch.tensor([true_label])
        else:
            return torch.rand(len(list(df.columns))).unsqueeze(0).double(), \
                   artist_index[artists_names[random.choice(range(0,len(df)))]], \
                   torch.tensor([fake_label])

In [11]:
class DeepEmbed(nn.Module):

    def __init__(self, feature_size, lookup_size, embedding_dim, batch_size):
        super(DeepEmbed, self).__init__()
        # Should Input size be (None,1) ?         
        self.embeddings = nn.Embedding(lookup_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.head = nn.Linear(feature_size, 128)
        self.op_layer = nn.Linear(256, 1)
        self.batch_size = batch_size

    def forward(self, feature_input, lookup_input):
        embeds = self.embeddings(lookup_input).view(self.batch_size, 1, -1)
        out1 = F.relu(self.linear1(embeds))
        out2 = F.relu(self.head(feature_input))
        output_concat = torch.cat([out1, out2], axis =-1)
        out = torch.sigmoid(self.op_layer(output_concat))
        return out.squeeze(1)

In [12]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [13]:
truth_set = Dataset(df, list(df.columns), is_truth=True)
fake_set = Dataset(df, list(df.columns), is_truth=False)
full_ds = torch.utils.data.ConcatDataset([truth_set, fake_set])
# training_generator = torch.utils.data.DataLoader(training_set, **params)

In [14]:
lookup_size = len(artists_names.unique())
emb_dim = 50
feature_size = len(df.columns)
learning_rate = 1e-4
batch_size = 256
epochs = 200 

criterion = nn.BCELoss()

In [15]:
train_dl = torch.utils.data.DataLoader(full_ds, batch_size=batch_size, shuffle=True, num_workers=0)
_iter = iter(train_dl)

In [16]:
features, artist, label = next(_iter)
# (features.shape), artist, index_artist[int(artist)] , label
features.shape

torch.Size([256, 1, 14])

In [17]:
model = DeepEmbed(feature_size = feature_size, lookup_size=lookup_size, \
                  embedding_dim=emb_dim, batch_size=batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(model)

DeepEmbed(
  (embeddings): Embedding(31465, 50)
  (linear1): Linear(in_features=50, out_features=128, bias=True)
  (head): Linear(in_features=14, out_features=128, bias=True)
  (op_layer): Linear(in_features=256, out_features=1, bias=True)
)


In [18]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    y_true = []
    y_pred = []
    for epoch in range(epochs):
        if epoch %25 == 0 :
            print(f"--EPOCH {epoch}--")
        for i in train_loader:
            feature, artist, label = i
            feature, artist, label = feature.cuda(), artist.cuda(),label.cuda()
            feature = feature.float()
            label = label.float()
            output = model(feature, artist)
            output = output

            loss = criterion(output, label) 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if epoch %25 == 0 :
                print(f"{loss.item()}")
            break

In [19]:
train_model = False

In [20]:
if train_model == True:
    train(model.cuda(), device, train_dl, optimizer, criterion)
else:
    model.load_state_dict(torch.load(model_path/'first_model.ckpt' ))

In [21]:
# torch.save(model.state_dict(),model_path/'first_model.ckpt' )

In [22]:
emb = model.embeddings

In [23]:
emb(torch.tensor([1]))

tensor([[-0.4301, -0.1413, -0.5302,  1.2199,  2.9091,  1.9989, -0.4521, -0.1614,
         -0.2575,  1.3220,  0.6549,  0.4241, -0.4755,  1.3491,  0.4467, -0.5560,
          0.1040, -1.0075,  0.7730,  0.9286, -0.6500,  0.2600, -1.2071,  1.0382,
          0.6945, -0.5230, -1.6711, -0.5926,  1.6206, -0.6386,  1.1328, -0.5365,
          0.4495,  1.6935, -0.4167, -0.6155,  2.1285,  1.1144, -0.6684,  1.1695,
         -0.5057,  2.2098, -0.1938,  0.8068, -1.2424,  1.1053,  1.5306, -0.2038,
          0.1986,  0.9018]], grad_fn=<EmbeddingBackward>)

In [24]:
d = dict() 
for i in range(1, len(artist_index)):
    d[index_artist[i]] = emb(torch.tensor([i])).detach().numpy()

In [26]:
with open(model_path/ 'deep_embeddings.pkl', 'wb') as handle:
    pkl.dump(d, handle, protocol=pkl.HIGHEST_PROTOCOL)